In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

In [2]:
inputs = torch.randn(1, 5, 5, 5)#(batch size, channels, width, heights)
filters = torch.randn(1, 5, 3, 3)
padding = 0
stride = 1
k_s = [3,3]

### Testing with torch

In [3]:
s = torch.nn.functional.unfold(inputs, kernel_size = [3,3], dilation=1, padding=padding, stride=stride)

In [4]:
s.shape

torch.Size([1, 45, 9])

In [5]:
s

tensor([[[ 1.3264, -0.3697, -1.6581,  0.6706,  1.2048,  0.4885,  0.9439,
           0.0943,  0.8263],
         [-0.3697, -1.6581, -0.3083,  1.2048,  0.4885, -2.9555,  0.0943,
           0.8263, -0.6674],
         [-1.6581, -0.3083, -0.4791,  0.4885, -2.9555,  2.1681,  0.8263,
          -0.6674,  1.5364],
         [ 0.6706,  1.2048,  0.4885,  0.9439,  0.0943,  0.8263,  0.3169,
           1.0148,  0.9412],
         [ 1.2048,  0.4885, -2.9555,  0.0943,  0.8263, -0.6674,  1.0148,
           0.9412, -1.6938],
         [ 0.4885, -2.9555,  2.1681,  0.8263, -0.6674,  1.5364,  0.9412,
          -1.6938,  1.4526],
         [ 0.9439,  0.0943,  0.8263,  0.3169,  1.0148,  0.9412,  0.8157,
          -0.6004, -0.4014],
         [ 0.0943,  0.8263, -0.6674,  1.0148,  0.9412, -1.6938, -0.6004,
          -0.4014,  0.0529],
         [ 0.8263, -0.6674,  1.5364,  0.9412, -1.6938,  1.4526, -0.4014,
           0.0529,  0.4170],
         [-0.5856,  0.4748,  1.1451, -1.7512, -0.1551, -0.9526,  0.8463,
         

### Our function 

In [6]:
filters.flatten()

tensor([ 0.7679, -1.3931, -0.8109, -0.0700, -1.0485, -1.3339,  0.4676, -0.1381,
        -1.8544, -1.3904, -0.5720, -0.5343, -0.4453, -0.5307, -0.0189, -1.0950,
         0.4646,  0.5704,  0.9090,  0.2847,  0.7382, -0.3006,  0.4597,  0.8318,
         0.6261,  0.5248,  0.2608,  0.6085,  1.1327, -1.7168, -0.4176,  0.0575,
         0.7769,  0.0413,  0.4228,  0.0256,  1.2902,  0.1741, -1.4666,  1.6896,
         0.0564,  0.5315, -1.0201, -0.8633, -0.4699])

In [7]:
inputs

tensor([[[[ 1.3264, -0.3697, -1.6581, -0.3083, -0.4791],
          [ 0.6706,  1.2048,  0.4885, -2.9555,  2.1681],
          [ 0.9439,  0.0943,  0.8263, -0.6674,  1.5364],
          [ 0.3169,  1.0148,  0.9412, -1.6938,  1.4526],
          [ 0.8157, -0.6004, -0.4014,  0.0529,  0.4170]],

         [[-0.5856,  0.4748,  1.1451, -0.3170,  0.8341],
          [-1.7512, -0.1551, -0.9526, -2.5395, -1.6424],
          [ 0.8463,  1.3558, -1.4581, -0.5908,  1.0450],
          [ 0.6962,  0.2564, -1.5092, -0.2837, -0.8746],
          [-0.1120,  0.6521, -0.2624, -1.0113,  1.3304]],

         [[-0.2773, -0.8155,  2.0539,  0.6695,  0.7360],
          [ 0.1143, -0.4924, -0.0491, -0.2102,  0.5179],
          [-0.5074,  0.3079,  0.6117, -0.4564,  0.7539],
          [-0.1960,  0.7214, -0.4631, -0.2587, -0.7078],
          [-1.3061, -0.6921,  0.0647, -1.0770,  0.0163]],

         [[-0.0592, -1.3736, -0.3351, -0.8329, -0.3126],
          [ 0.2177, -0.3167, -0.6304, -0.4151,  0.8808],
          [-0.4338,  0.16

In [8]:
inputs.shape

torch.Size([1, 5, 5, 5])

In [9]:
patches = []
# vec_y = []
p1 = int(((inputs.shape[2] + (2*padding) - k_s[0])/stride) +1)
p2 = int(((inputs.shape[3] + (2*padding) - k_s[1])/stride) +1)
p = p1*p2
counter=0 
temp = []
inputs = np.pad(inputs, pad_width = ((0,0),(0,0),(padding,padding),(padding,padding)))
for b in range(inputs.shape[0]):
    for c in range(inputs.shape[1]): 
        for h in range(0,inputs.shape[2],stride):
            for w in range(0,inputs.shape[3],stride):
                vec_x = inputs[b][c][h:k_s[0]+h,:][:,w:k_s[1]+w]
                if (vec_x.shape[0]==k_s[0] & vec_x.shape[1]==k_s[1]):
                    vec_x = vec_x.flatten()
                    patches.append(vec_x)
                    
vec_y = np.array(patches) * np.array(filters.flatten()[:, np.newaxis])

In [10]:
vec_y.shape

(45, 9)

In [11]:
patches

[array([ 1.3264308 , -0.36965796, -1.6581029 ,  0.6705813 ,  1.2048064 ,
         0.4885479 ,  0.94386715,  0.09427062,  0.8262826 ], dtype=float32),
 array([-0.36965796, -1.6581029 , -0.30831492,  1.2048064 ,  0.4885479 ,
        -2.9554873 ,  0.09427062,  0.8262826 , -0.66744757], dtype=float32),
 array([-1.6581029 , -0.30831492, -0.4791309 ,  0.4885479 , -2.9554873 ,
         2.168116  ,  0.8262826 , -0.66744757,  1.5364361 ], dtype=float32),
 array([0.6705813 , 1.2048064 , 0.4885479 , 0.94386715, 0.09427062,
        0.8262826 , 0.31691375, 1.0147967 , 0.9411889 ], dtype=float32),
 array([ 1.2048064 ,  0.4885479 , -2.9554873 ,  0.09427062,  0.8262826 ,
        -0.66744757,  1.0147967 ,  0.9411889 , -1.6937854 ], dtype=float32),
 array([ 0.4885479 , -2.9554873 ,  2.168116  ,  0.8262826 , -0.66744757,
         1.5364361 ,  0.9411889 , -1.6937854 ,  1.4526248 ], dtype=float32),
 array([ 0.94386715,  0.09427062,  0.8262826 ,  0.31691375,  1.0147967 ,
         0.9411889 ,  0.8157408 , -0